The goal is to create a database of earning as a driver to compare over time, the average wage per day. This will serve as a means to judge just how much this short term change in occupation  will affect me monetarily.

# Thoughts / Plan / Notes

- set up dictionary of lists to hold info so that per day each item is ordered?
- estabilish a wage
- make a list / record of earnings per day. 
- Final earnings need to be (wage * hours)+tips)
- effective hourly wage = earnings / hours 
- profit = earnings * tax rate (0.8225)
- set up comparison equation to display earnings at old job amount and effctive change per day
- connect to a db?

> End goal
>> an application that request info, stores to db, then displays plotted changes / info over time.

- estabilish means of recording info
- process the info
- record in a way that allows querying and graphing for data viz purposes
- display information

## Notes

Should the index be the day? How to format?

need to add means of making sure dates are a certain length, if you mess up the date, theres no fixing > the entry must be scrapped. ALSO messing up the entry breaks the code.


--------------------------------------------------------------------------------------

## Global Values - Run *once*

In [1]:
import pymongo

# set up pymongo connection

# https://docs.mongodb.com/manual/reference/default-mongodb-port/

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare database
db = client.jj_earnings

# declare the collection
shifts = db.shifts
test = db.test

current_wage = 9.5 
chipotle_wage = 16.5
#-------------------------------------------------------------------------------------------------------------------------#


def get_date():
    # gets the date in a no frills string, formats to desired output for later use
    # returns variable "date_of_shift"
    # breaks if date != 6 characters
    print("Use 2 characters for the Day, Month, and Year.")
    print("EXAMPLE: 01/15/2022 = 011522")
    print("")
    
    date_string = input("Please input date of shift. ")
    
    if len(date_string) != 6:
        raise ValueError ('Date Incorrect Format')
    else:
        month = date_string[slice(0,2)]
        day = date_string[slice(2,4)]
        year = date_string[slice(4,6)]
        date_of_shift = f'{month}-{day}-{year}'
        return date_of_shift

#-------------------------------------------------------------------------------------------------------------------------#

def earnings_request():
    
    # asks for information and returns the date & dict of aggregated values
    wage = current_wage
    date_of_shift = get_date()
    print("")
    hours = float(input("Hours worked? "))
    if hours == 0:
        raise ValueError('something something, div by zero is bad.')
    else:
    
        tips = float(input("Tips? "))
        start_mileage = input("What was the starting mileage? ")
        end_mileage = input("What was the ending mileage? ")
        start_mileage = int(start_mileage)
        end_mileage = int(end_mileage)
        delta_mileage = end_mileage - start_mileage


        earnings = round(((wage * hours) + tips),2)
        effective_wage = round((earnings / hours), 2)
    
        old_job_earnings = round((chipotle_wage * hours),2)
        old_job_diff = old_job_earnings - earnings

        entry = {
        "date_of_shift":date_of_shift,
        "tips": tips,
        "hours":hours,
        "earnings":earnings,
        "effective_wage":effective_wage,
        "old_job_earnings":old_job_earnings,
        "old_job_comparison":old_job_diff,
        "starting_mileage": start_mileage,
        "ending_mileage": end_mileage,
        "change_in_mileage": delta_mileage,
        
        }


        return entry

        
#-------------------------------------------------------------------------------------------------------------------------#   return date_of_shift

def main_function():

    finished = 'n'

    while finished != 'y':
        value = True
        while value == True:
            try:
                earnings_data = {}
                shift_data = earnings_request()
                
                earnings_data = shift_data
                print('--------------------------------------')
                print('Please review info, enter "cancel" to restart. ')
                finished = input('Done? (y/n) ').lower()
                print('--------------------------------------')
                if finished == 'cancel':
                    raise ValueError
                else:

                    print('Sending document to jj_earnings in MongoDB...')
                    print('--------------------------------------')
                    print()
                    shifts.insert_one(earnings_data)
                    value = False
                
            except ValueError:
                print('')
                print('====================================')
                print("Incorrect Data Entry, try that again.")
                print('====================================')
                print('')
#-------------------------------------------------------------------------------------------------------------------------#


------------------------------------------------------------------

In [2]:
main_function()

Use 2 characters for the Day, Month, and Year.
EXAMPLE: 01/15/2022 = 011522

Please input date of shift. 011422

Hours worked? 8.5
Tips? 99
What was the starting mileage? 75575
What was the ending mileage? 75640
--------------------------------------
Please review info, enter "cancel" to restart. 
Done? (y/n) y
--------------------------------------
Sending document to jj_earnings in MongoDB...
--------------------------------------



now that we have a method of capturing the information, we need to send it to a database

NOTE - we need a method of catching the data entries should they not match the desired input
eg - date having only one character input vs the full six.

implement more anti-breaking  / NaN data entry methods and fail-safes to save time should errors arise.


make a method to querydb, select doc and edit information before saving over.